In [15]:
import numpy as np
from pyfigtree import figtree
from scipy.optimize import fsolve

In [2]:
def position(X,alpha,delta):
    X_x = figtree(X,X,alpha[:,0], delta,epsilon=1e-16, eval="auto")
    X_y = figtree(X,X,alpha[:,1], delta,epsilon=1e-16, eval="auto")
    X_z = figtree(X,X,alpha[:,2], delta,epsilon=1e-16, eval="auto") 
    return np.dstack([X_x,X_y, X_z])[0]

In [3]:
def momentum(X,alpha,delta):
    ax = alpha[:,0]
    ay = alpha[:,1]
    az = alpha[:,2]
    
    first_weights = np.dstack([ax * X[:,0], ax * X[:,1], ax * X[:,2]])[0]
    second_weights = np.dstack([ay * X[:,0], ay * X[:,1], ay * X[:,2]])[0]
    third_weights = np.dstack([az * X[:,0], az * X[:,1], az * X[:,2]])[0]
    
    first_sum_x = figtree(X,X,first_weights[:,0],delta, epsilon=1e-16, eval="auto")
    first_sum_y = figtree(X,X,first_weights[:,1],delta, epsilon=1e-16, eval="auto")
    first_sum_z = figtree(X,X,first_weights[:,2],delta, epsilon=1e-16, eval="auto")
    
    
    second_sum_x = figtree(X,X,second_weights[:,0],delta, epsilon=1e-16, eval="auto")
    second_sum_y = figtree(X,X,second_weights[:,1],delta, epsilon=1e-16, eval="auto")
    second_sum_z = figtree(X,X,second_weights[:,2],delta, epsilon=1e-16, eval="auto")
    
    
    third_sum_x = figtree(X,X,third_weights[:,0],delta, epsilon=1e-16, eval="auto")
    third_sum_y = figtree(X,X,third_weights[:,1],delta, epsilon=1e-16, eval="auto")
    third_sum_z = figtree(X,X,third_weights[:,2],delta, epsilon=1e-16, eval="auto")
    
    fourth_sum = figtree(X,X,ax,delta, epsilon=1e-16, eval="auto")
    fifth_sum = figtree(X,X,ay,delta, epsilon=1e-16, eval="auto")
    sixth_sum = figtree(X,X,az,delta, epsilon=1e-16, eval="auto")
    
    second_term = ax * fourth_sum + ay * fifth_sum + az * sixth_sum
    
    second_term_x = X[:,0] * second_term
    second_term_y = X[:,1] * second_term
    second_term_z = X[:,2] * second_term
 
    mult = -2./(delta ** 2)
    result_x = mult*(ax* first_sum_x + ay * second_sum_x + az * third_sum_x - (second_term_x) )
    result_y = mult*(ax* first_sum_y + ay * second_sum_y + az * third_sum_y - (second_term_y) )
    result_z = mult*(ax* first_sum_z + ay * second_sum_z + az * third_sum_z - (second_term_z) )
    result = np.dstack([result_x,result_y,result_z])[0]
    
    return result

In [38]:
def f(vector, delta):
    positions, momentums = np.split(vector,2)
    positions = np.reshape(positions,(3,len(positions)/3)).T
    momentums = np.reshape(momentums,(3,len(momentums)/3)).T
    new_positions = position(positions, momentums,delta)
    new_momentums = momentum(positions, momentums,delta)
    n,m = new_positions.shape
    new_positions = np.reshape(new_positions.T, (n*m))
    new_momentums = np.reshape(new_momentums.T, (n*m))
    return np.hstack([new_positions,new_momentums])

In [245]:
# Intento de primer jacobiano
def df1dx(X, alpha, sigma):
    mult = -2/(sigma ** 2)
    
    X_x, X_y, X_z = np.split(X, 3)
    X_x = X_x.reshape(1, X_x.shape[0])
    X_y = X_y.reshape(1, X_y.shape[0])
    X_z = X_z.reshape(1, X_z.shape[0])
    
    a_x, a_y, a_z = np.split(alpha, 3)
    a_x = a_x.reshape(1, a_x.shape[0])
    a_y = a_y.reshape(1, a_y.shape[0])
    a_z = a_z.reshape(1, a_z.shape[0])
    
    ones = np.ones((1, X_x.shape[1]))
    
    X_x_square = X_x * ones.T
    X_y_square = X_y * ones.T
    X_z_square = X_z * ones.T
    
    X_x_diff = X_x_square - X_x_square.T
    X_y_diff = X_y_square - X_y_square.T
    X_z_diff = X_z_square - X_z_square.T
    
    X_diff = np.sqrt(X_x_diff ** 2 + X_y_diff ** 2 + X_z_diff ** 2)
    
    exp = np.exp(-(X_diff ** 2)/(sigma ** 2))
    
    dX_x = exp * X_x_diff * a_x
    dX_y = exp * X_y_diff * a_y
    dX_y = exp * X_y_diff * a_y
    
    print dX_x
    

In [247]:
df1dx(X0, alpha0, delta)

[[ -0.00000000e+00   8.06407601e-09  -3.75531476e-05 ...,   2.20768987e-02
    7.33692313e-03   1.82376408e-02]
 [ -2.92703952e-07  -0.00000000e+00   3.75531476e-05 ...,  -9.48442816e-06
   -6.15718881e-06  -7.83504949e-06]
 [ -3.13217641e-04   8.62923391e-06   0.00000000e+00 ...,  -5.35514472e-06
   -1.42741659e-06  -2.92221335e-06]
 ..., 
 [ -1.21476601e-02   1.43777771e-07  -3.53285577e-07 ...,  -0.00000000e+00
    2.85923544e-02  -0.00000000e+00]
 [ -9.09507871e-03   2.10281463e-07  -2.12150159e-07 ...,  -6.44151235e-02
   -0.00000000e+00  -5.32130714e-02]
 [ -1.21476629e-02   1.43777804e-07  -2.33365216e-07 ...,  -0.00000000e+00
    2.85923538e-02  -0.00000000e+00]]


In [248]:
vector0 = np.hstack((X0, alpha0))
diff = f(vector0, delta)
dx, dm = np.split(diff,2)

alpha0 = np.reshape(alpha0,(3,len(alpha0)/3)).T
print "momentum0", np.sum(alpha0[:,0]), np.sum(alpha0[:,1]), np.sum(alpha0[:,2])
print "momentum0", (np.sum(alpha0[:,0]) ** 2 + np.sum(alpha0[:,1]) ** 2 + np.sum(alpha0[:,2]) ** 2) ** 0.5

dm = np.reshape(dm,(3,len(dm)/3)).T
print np.sum(dm, axis=0)
alpha1 = alpha0 + (dm * 0.01)
X1 = X0 + (dx * 0.01)
print "momentum1", np.sum(alpha1[:,0]), np.sum(alpha1[:,1]), np.sum(alpha1[:,2])
print "momentum1", (np.sum(alpha1[:,0]) ** 2 +  np.sum(alpha1[:,1]) ** 2 + np.sum(alpha1[:,2]) ** 2) ** 0.5
print X0
print X1
print X1- X0

momentum0 -29.746298 -33.989227 87.78351
momentum0 98.7221070717
[  2.43804976e-13   3.39284156e-13   1.22568622e-13]
momentum1 -29.746298 -33.989227 87.78351
momentum1 98.7221070717
[ 65.949541  29.949541  47.949541 ...,  39.114004  39.114004  40.087333]
[ 65.94656081  29.8918754   47.95193877 ...,  39.13619889  39.13314077
  40.114768  ]
[-0.00298019 -0.0576656   0.00239777 ...,  0.02219489  0.01913677  0.027435  ]


In [83]:
sigma = 6.5 * (2**0.5)

X = np.loadtxt('data/sphere_x.txt')
Y = np.loadtxt('data/sphere_y.txt')
Z = np.loadtxt('data/sphere_z.txt')

X0 = np.hstack((X,Y,Z))

XF = np.loadtxt('data/vent_x.txt')
YF = np.loadtxt('data/vent_y.txt')
ZF = np.loadtxt('data/vent_z.txt')

X1 = np.hstack((XF,YF,ZF))

A = np.loadtxt('data/alpha0_x.txt')
B = np.loadtxt('data/alpha0_y.txt')
C = np.loadtxt('data/alpha0_z.txt')

alpha0 = np.hstack((A,B,C))

In [84]:
def shoot(alpha, a, b, x_a, x_b, h, sigma):
    x_i = x_a
    a_i = alpha
    print "alpha", np.sum(np.reshape(alpha0,(3,len(alpha0)/3)).T, axis = 0)
    for i in np.arange(a, b + h, h):
        vector = np.hstack((x_i, a_i))
        diff = f(vector, sigma)
        dx, dm = np.split(diff,2)
        x_i_next = x_i + dx * h
        a_i_next = a_i + dm * h
        x_i = x_i_next
        a_i = a_i_next
    print np.sum(x_i - x_b)/np.sum(x_b)
    return x_i - x_b

In [85]:
def shootingMethod(a, b, x_a, x_b, alpha0, h, sigma):
    alpha = fsolve(shoot, alpha0, (a, b, x_a, x_b, h, sigma))
    return alpha

In [87]:
alpha = shootingMethod(0, 1, X0, X1, alpha0, 0.5, sigma)

alpha [-29.746298 -33.989227  87.78351 ]
0.00333273251199
alpha [-29.746298 -33.989227  87.78351 ]
0.00333273251199
alpha [-29.746298 -33.989227  87.78351 ]


KeyboardInterrupt: 